<a href="https://colab.research.google.com/github/ClaudeCoulombe/VIARENA/blob/master/Labos/Lab-Identification_Arbres/Identification_arbre_reseau_convolutif_et_transfert_colab.ipynb" target="_blank"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Rappel - Fonctionnement d'un carnet web iPython

* Pour exécuter le code contenu dans une cellule d'un carnet iPython, cliquez dans la cellule et faites (⇧↵, shift-enter) 
* Le code d'un carnet iPython s'exécute séquentiellement de haut en bas de la page. Souvent, l'importation d'une bibliothèque Python ou l'initialisation d'une variable est préalable à l'exécution d'une cellule située plus bas. Il est donc recommandé d'exécuter les cellules en séquence. Enfin, méfiez-vous des retours en arrière qui peuvent réinitialiser certaines variables.

# Identification d'arbres à partir de leur écorce
## Réseau convolutif et apprentissage par transfert

##### Copyright 2019 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

## Apprentissage par transfert et paufinage

In [ ]:
import os

In [ ]:
# Note that we did not use Acer platanoides (2), Pinus rigida (15) and Populus grandidentata (18)
# since we did not collect enough images in these categories to obtain meaningful results.
data_ecorces = {
    'SAB': 1,  # Pas assez d'espace disque
    'ERB': 2,  # Pas assez de spécimens - seulement 1
    'ERR': 3, 
    'ERS': 4, 
    'BOJ': 5, 
    'BOP': 6,
    'HEG': 7,  # Pas assez d'espace disqe
    'FRA': 8, 
    'MEL': 9,  # Pas assez d'espace disque
    'OSV': 10, # Pas assez d'espace disque
    'EPO': 11,
    'EPB': 12,
    'EPN': 13,
    'EPR': 14,
    'PID': 15, # Pas assez de spécimens - seulement 4
    'PIR': 16, # Pas assez d'espace disque
    'PIB': 17, # Pas assez d'espace disque
    'PEG': 18, # Pas assez de spécimens - seulement 3
    'PET': 19, # Pas assez d'espace disque
    'CHR': 20,
    'THO': 21, # Pas assez d'espace disque
    'PRU': 22, # Pas assez d'espace disque
    'ORA': 23  # Pas assez d'espace disque
}

noms_arbres = {
            1: '\emph{Abies balsamea} - Sapin Baumier - Balsam fir',
            2: '\emph{Acer platanoides} - Érable de Norvège - Norway maple',
            3: '\emph{Acer rubrum} - Érable rouge - Red maple',
            4: '\emph{Acer saccharum} - Érable à sucre - Sugar maple',
            5: '\emph{Betula alleghaniensis} - Bouleau jaune - Yellow birch',
            6: '\emph{Betula papyrifera} - Bouleau à papier - White birch',
            7: '\emph{Fagus grandifolia} - Hêtre à grandes feuilles - American beech',
            8: "\emph{Fraxinus americana} - Frêne d'Amérique - White ash",
            9: '\emph{Larix laricina} - Mélèze - Tamarack',
            10: '\emph{Ostrya virginiana} - Ostryer de Virginie - American hophornbeam',
            11: '\emph{Picea abies} - Épinette de Norvège - Norway spruce',
            12: '\emph{Picea glauca} - Épinette blanche - White spruce',
            13: '\emph{Picea mariana} - Épinette noire - Black spruce',
            14: '\emph{Picea rubens} - Épinette rouge - Red spruce',
            15: '\emph{Pinus rigida} - Pin rigide - Pitch pine',
            16: '\emph{Pinus resinosa} - Pin rouge - Red pine',
            17: '\emph{Pinus strobus} - Pin blanc - Eastern white pine',
            18: '\emph{Populus grandidentata} - Peuplier à grandes dents - Big-tooth aspen',
            19: '\emph{Populus tremuloides} - Peuplier faux tremble - Quaking aspen',
            20: '\emph{Quercus rubra} - Chêne rouge - Northern red oak',
            21: '\emph{Thuja occidentalis} - Thuya occidental - Northern white cedar',
            22: '\emph{Tsuga canadensis} - Pruche du Canada - Eastern Hemlock',
            23: "\emph{Ulmus americana} - Orme d'Amérique - American elm"
        }
print("Code executed")

In [ ]:
data_zip_urls_dict = {
   "BOJ":"https://drive.google.com/file/d/1d2zxg2pt5S8UJIK-E7IuWfGN0d1kxxMw/view?usp=sharing",
   "BOP":"https://drive.google.com/file/d/12cg6UO4HLnjk5fE_KXtrgdC2s8uGh4Zp/view?usp=sharing",
   "CHR":"https://drive.google.com/file/d/1Nq19-I-Q577KXMTFrkhlJDhMfclh0cWn/view?usp=sharing",
   "EPB":"https://drive.google.com/file/d/1K_Ncw8VEiuDZ_iJDbYToMq-GO5dzKHns/view?usp=sharing",
   "EPN":"https://drive.google.com/file/d/1S309DYmg76SrIA89aVQWXCMwm6CzhN8b/view?usp=sharing",
   "EPO":"https://drive.google.com/file/d/1fTKEcpYgmRg4spUpcH0FAiAnoRgANafL/view?usp=sharing",
   "EPR":"https://drive.google.com/file/d/1qRhtZ8LZjH_45fxetG7swg3ok3znk8CJ/view?usp=sharing",
   "ERB":"https://drive.google.com/file/d/1ighbGniKAT_GrPm4RtsIAuN1STg9sjR9/view?usp=sharing", # Assez de données?
   "ERR":"https://drive.google.com/file/d/1rEo1thMNJTgFeTzTOfI11_FPSqMgbHSL/view?usp=sharing",
   "ERS":"https://drive.google.com/file/d/1ts-t7bOH9DfKj0q0v35nMgKHgVT0ZjyG/view?usp=sharing",
   "FRA":"https://drive.google.com/file/d/1yLacRGW7JtlFWV5asEXHpAToClL38D64/view?usp=sharing",
   "HEG":"https://drive.google.com/file/d/1zoJKEIrsCD1XxglgPJkEygumev1xRQ3U/view?usp=sharing",
   "MEL":"https://drive.google.com/file/d/1Wdy3DDnWfUysXjcIFFq12UFW7tlTYDT2/view?usp=sharing",
   "ORA":"https://drive.google.com/file/d/19_oYwCAaPfP6vMuqUnAzIQAa39Brxhfi/view?usp=sharing",
   "OSV":"https://drive.google.com/file/d/1VJCCZN1iwBK2Nzh_PHC9xvw63xiLuXXI/view?usp=sharing",
   "PEG":"https://drive.google.com/file/d/1YUWH4IaTnmcoIAavZq8HyXByJxO7_zBg/view?usp=sharing", # Assez de données?
   "PET":"https://drive.google.com/file/d/13bMkvr_1mRz1TuOcX8-c-LfTSIsNKrve/view?usp=sharing",
   "PIB":"https://drive.google.com/file/d/13bMkvr_1mRz1TuOcX8-c-LfTSIsNKrve/view?usp=sharing",
   "PID":"https://drive.google.com/file/d/12xswrf4pDmTAcYZDAY9D-0HniLjGJCxp/view?usp=sharing", # Assez de données?
   "PIR":"https://drive.google.com/file/d/1qny4meuoT-HYZ_KTyPQbQnzLhebkgkfU/view?usp=sharing",
   "PRU":"https://drive.google.com/file/d/1xQWHQvIbwRRBoi2F27q22_drUeM8m3S8/view?usp=sharing",
   "SAB":"https://drive.google.com/file/d/1ol2mlYAz5bMfQkwqcnxhCOg4avftYtRe/view?usp=sharing",
   "THO":"https://drive.google.com/file/d/1_mI0saGpfxb4wnhElCzxg0WU4OiFHkfP/view?usp=sharing",
  
}
data_zip_urls_dict

In [ ]:
# Création des répertoires de données
# Nous allons créer un répertoire de base `src` et des répertoiresnpour les données 
# d'entrainement, de validation et de test pour chaque étiquette cible

try:
    os.mkdir("/tmp/src/")
except OSError:
    pass
try:
    os.mkdir("/tmp/lab_id_arbres/")
except OSError:
    pass


In [ ]:
# Référence: https://colab.research.google.com/notebooks/io.ipynb
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import os
import shutil
import zipfile


In [ ]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
nbr_classes = 0
for arbre_id in data_zip_urls_dict.keys():
    url = data_zip_urls_dict[arbre_id]
    id_fichier = url.split('/')[5]
    fichier = drive.CreateFile({'id':id_fichier})
    nom_fichier = arbre_id + ".zip"
    # télécharger le fichier nom_fichier
    fichier.GetContentFile("/tmp/src/" + nom_fichier)
    print("Fichier " + nom_fichier + " téléchargé")
    zip_ref = zipfile.ZipFile("/tmp/src/" + nom_fichier, 'r')
    zip_ref.extractall("/tmp/src")
    zip_ref.close()
    print("Fichier " + nom_fichier + " décompressé")
    try:
        os.remove("/tmp/src/"+nom_fichier)
        print("Fichier " + nom_fichier + " effacé")
    except:
        print("?")
    nbr_classes += 1
shutil.rmtree('/tmp/src/__MACOSX')
print("nbr_classes:",nbr_classes)

In [ ]:
!pip3 install split-folders tqdm

In [ ]:
# Répartition des données d'entraînement et de tests
import splitfolders

#### input dataset that want to split
input_folder = "/tmp/src"
output_folder= "/tmp/lab_id_arbres"
# => train, val, test

splitfolders.ratio(input_folder, output= output_folder, seed=42, ratio = (0.95, 0.0, 0.05))

print("Répartition des données terminée!")

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
import matplotlib.image as mpimg

pic_index = 4

TRAINING_DIR = "/tmp/lab_id_arbres/train/"

for arbre_id in data_ecorces.keys():
    try:
        dir_path = os.path.join(TRAINING_DIR,arbre_id+os.sep)
        liste_fichiers = os.listdir(os.path.join(TRAINING_DIR,arbre_id+os.sep))
        next_two_pics = [os.path.join(dir_path, fname) for fname in liste_fichiers[0:pic_index]]
        fig = plt.figure(figsize=(12,4))
        print("_"*90)
        print(arbre_id,noms_arbres[data_ecorces[arbre_id]].split('-')[1])
        for i, img_path in enumerate(next_two_pics):
            print(img_path)
            img = mpimg.imread(img_path)
            plt.subplot(1,pic_index,i+1)
            plt.imshow(img)
            plt.axis('Off')
        plt.show()
    except:
        continue

# Création d'un modèle d'apprentissage par transfert

## Importation d'un modèle inception pré-entraîné

In [ ]:
!wget --no-check-certificate \
    https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5 \
    -O /tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5

In [ ]:
import os

import keras
print("Keras version:",keras.__version__)
import tensorflow as tf
print("TensorFlow version:",tf.__version__)


from tensorflow.keras import layers
from tensorflow.keras import Model

from tensorflow.keras.applications.inception_v3 import InceptionV3

pre_trained_model = InceptionV3(weights='imagenet',  # Load weights pre-trained on ImageNet.
                                input_shape = (150, 150, 3), 
                                include_top = False)

# local_weights_file = '/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'
# pre_trained_model.load_weights(local_weights_file)

pre_trained_model.trainable = True

# Let's take a look to see how many layers are in the pretrained model
nbr_layers_pretrained_model = len(pre_trained_model.layers)
print("Nombre de couches dans le modèle pré-entraîné d'origine: ", nbr_layers_pretrained_model)

# How to get the layer index from the layer name
# https://www.thetopsites.net/article/50151157.shtml
layer_names = [layer.name for layer in pre_trained_model.layers]
last_layer_name = layer_names[-1]
print("Nom de la dernière couche du modèle pré-entraîné complet:",last_layer_name)
last_layer_index = layer_names.index(last_layer_name)
# Choix d'une nouvelle couche de sortie par essai / erreur
last_layer_name = 'mixed5'
print("Choix empirique de la dernière du modèle pré-entraîné:",last_layer_name)
last_layer_index = layer_names.index(last_layer_name)
print("Index de la dernière couche du modèle pré-entraîné:",last_layer_index)
# Choix de la dernière couche non-entraînable ou dernière couche « gelée »
last_layer_frozen_name = 'mixed3'
print("Choix empirique de la dernière couche non-entraînable:",last_layer_frozen_name)
last_layer_frozen_index = layer_names.index(last_layer_frozen_name)
print("Index de la dernière couche non-entraînable:",last_layer_frozen_index)
# Fine-tune from this layer onwards
fine_tune_at = last_layer_frozen_index
print("Nombre de couches non entraînables dans le modèle préentraîné: ", fine_tune_at )
# Freeze all the layers before the `fine_tune_at` layer
for layer in pre_trained_model.layers[:fine_tune_at]:
    layer.trainable =  False

print("Nombre de couches entraînables dans le modèle préentraîné: ", last_layer_index-fine_tune_at)
last_layer = pre_trained_model.get_layer(last_layer_name)
print('Dimensions de la dernière couche: ', last_layer.output_shape)
last_output = last_layer.output


In [ ]:
# Create new data input
inputs = keras.Input(shape=(150, 150, 3))
#inputs = data_augmentation(inputs)  # Apply random data augmentation

# Pre-trained Xception weights requires that input be scaled
# from (0, 255) to a range of (-1., +1.), the rescaling layer
# outputs: `(inputs * scale) + offset`

# data_augmentation = keras.Sequential(
#     [layers.RandomFlip("horizontal"), layers.RandomRotation(0.1),]
# )
# inputs = data_augmentation(inputs)  # Apply random data augmentation

scale_layer = keras.layers.Rescaling(scale=1/127.5,offset=-1)
inputs = scale_layer(inputs)

pre_trained_model.inputs = inputs

In [ ]:
# Flatten the output layer to 1 dimension
outputs = layers.Flatten()(last_output)
# Add a fully connected layer with 1,024 hidden units and ReLU activation
outputs = layers.Dense(1024, activation='relu')(outputs)
# Add a dropout rate of 0.5
outputs = layers.Dropout(0.5)(outputs)      
# # Add a bottleneck layer
# outputs = layers.Dense(512, activation='relu')(outputs)
# outputs = layers.Dropout(0.2)(outputs)  
# # Add a fully connected layer with 1,024 hidden units and ReLU activation
# outputs = layers.Dense(1024, activation='relu')(outputs)
# outputs = layers.Dropout(0.2)(outputs)  
# Add a final softmax layer for classification
# *** IMPORTANT *** 23 classes
number_of_target_class = 23
outputs = layers.Dense(number_of_target_class, activation='softmax')(outputs)           

transfer_model = Model(pre_trained_model.input, outputs) 
# transfer_model = Model(pre_trained_model.input, outputs)
print("Number of layers in the learning transfer model: ", len(transfer_model.layers))


In [ ]:
transfer_model.summary()


In [ ]:
len(transfer_model.trainable_variables)


In [ ]:
from tensorflow.keras.optimizers import RMSprop

# According to tf.keras.utils.image_dataset_from_directory documentation, 'int' is the default label_mode
# 'int': means that the labels are encoded as integers (e.g. for sparse_categorical_crossentropy loss)
# So, by default, tf.keras.utils.image_dataset_from_directory will create a set of labels for the dataset 
# as integer that go from 1 to the number of classes in the dataset.
# In this case, the model should be compiled with a 'sparse_categorical_crossentropy' loss.
transfer_model.compile(optimizer = RMSprop(learning_rate=0.0001), 
              loss = 'sparse_categorical_crossentropy', 
              metrics = ['accuracy'])

print("Modèle compilé!")

In [ ]:
TRAINING_DIR = "/tmp/lab_id_arbres/train/"
VALIDATION_DIR = "/tmp/lab_id_arbres/val"
TESTING_DIR = "/tmp/lab_id_arbres/test/"

BATCH_SIZE = 32
IMG_SIZE = (150, 150)

train_dataset = tf.keras.utils.image_dataset_from_directory(TRAINING_DIR,
                                                            shuffle=True,
                                                            validation_split=0.2,
                                                            subset='validation',
                                                            seed=42,
                                                            batch_size=BATCH_SIZE,
                                                            image_size=IMG_SIZE)

test_dataset = tf.keras.utils.image_dataset_from_directory(TESTING_DIR,
                                                           shuffle=True,
                                                           batch_size=BATCH_SIZE,
                                                           image_size=IMG_SIZE)

In [ ]:
print('Number of training batches: %d' % tf.data.experimental.cardinality(train_dataset))
print('Number of test batches: %d' % tf.data.experimental.cardinality(test_dataset))

In [ ]:
initial_epochs = 10
fine_tune_epochs = 10
total_epochs =  initial_epochs + fine_tune_epochs

# Callbacks are passed to the model via the callbacks argument in fit, 
# which takes a list of callbacks. You can pass any number of callbacks.
callbacks_list = [
    # # Interrupts training when improvement stops
    # keras.callbacks.EarlyStopping(
    #     # Monitors the model’s validation accuracy
    #     monitor='val_accuracy',
    #     # Interrupts training when accuracy has stopped 
    #     # improving for more than one epoch (that is, two epochs)
    #     patience=2,
    # ),
    keras.callbacks.ReduceLROnPlateau(
        # Monitors the model’s validation loss
        monitor='loss',
        # Divides the learning rate by 2 when triggered
        factor=0.5,
        # The callback is triggered after the validation loss 
        # has stopped improving for 1 epochs.
        patience=1,
    ) 

]
history_fine = transfer_model.fit(train_dataset,
                                  epochs = total_epochs,
                                  callbacks=callbacks_list,
                                  verbose = 2)


In [ ]:
import matplotlib.pyplot as plt
acc = history_fine.history['accuracy']
# val_acc = history_fine.history['val_accuracy']
loss = history_fine.history['loss']
# val_loss = history_fine.history['val_loss']
epochs = range(len(acc))
golden_number = 1.618
height = 6
length = int(golden_number * height)
plt.figure(figsize=(length,height))
plt.plot(epochs, acc, 'r', label='Training accuracy')
# plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy - augmentation on training data with dropout')
plt.legend(loc=0)
plt.xlabel("epoch")
plt.ylabel("accuracy")
plt.show()


In [ ]:
loss, accuracy = transfer_model.evaluate(test_dataset)
print('Test accuracy :', accuracy)

In [ ]:
print("Fin de l'exécution du carnet IPython")